In [0]:
import urllib
import urllib.parse as urlp
import requests
import json
import xml.etree.ElementTree as ET
import xmljson
#from google.colab import files
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os

In [0]:
#!pip install xmljson

In [0]:
url = "https://scihub.copernicus.eu/dhus/search?start=0&rows=100&q="
sensor = 'platformname:"Sentinel-2"'
footprint = 'footprint:"intersects(3.502020,-76.355062)"'
#ccp= 'cloudcoverpercentage:[0 TO 30]'
date = 'ingestiondate:[2018-07-26T00:00:00.000Z TO 2018-11-13T00:00:00.000Z]'

In [0]:
rurl = url + sensor + ' AND ' + footprint + ' AND ' + date
print(rurl)

https://scihub.copernicus.eu/dhus/search?start=0&rows=100&q=platformname:"Sentinel-2" AND footprint:"intersects(3.502020,-76.355062)" AND ingestiondate:[2018-07-26T00:00:00.000Z TO 2018-11-13T00:00:00.000Z]


In [0]:
r = requests.get(rurl, auth=('hamishgibbs', 'Hamish123!'))

In [0]:
from lxml.etree import fromstring, tostring
xml = bytes(bytearray(r.text, encoding = 'utf-8'))
xml = fromstring(xml)
json1 = json.dumps(xmljson.badgerfish.data(xml))

In [0]:
data = json.loads(json1)

In [0]:
records = data['{http://www.w3.org/2005/Atom}feed']['{http://a9.com/-/spec/opensearch/1.1/}totalResults']['$']
print('Number of Scenes: ' + str(records))
if records > 100:
  print('WARNING: More than 100 results, adjust paging options.')

Number of Scenes: 37


In [0]:
scenes = data['{http://www.w3.org/2005/Atom}feed']['{http://www.w3.org/2005/Atom}entry'][0:100]
df_cols = ['DATE','ID', 'NAME', 'QL_URL']
S2_df = pd.DataFrame(columns=df_cols)
for i, record in enumerate(scenes):
  date = record['{http://www.w3.org/2005/Atom}summary']['$'][5:16]
  dl_id = record['{http://www.w3.org/2005/Atom}id']['$']
  name = record['{http://www.w3.org/2005/Atom}title']['$']
  ql_url = record['{http://www.w3.org/2005/Atom}link'][2]['@href']
  S2_df.loc[i] = [date, dl_id, name, ql_url]
S2_df['DATE'] = S2_df['DATE'].astype('datetime64[ns]')

In [0]:
S2_df

,DATE,ID,NAME,QL_URL
0,2016-05-02,671de52d-c919-46dd-87cd-22dbb2ed0c9d,S2A_MSIL1C_20160502T153622_N0201_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
1,2016-05-09,bf7b165f-e3fc-4fbe-bbc4-64b63ca66d92,S2A_MSIL1C_20160509T152642_N0202_R025_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
2,2018-11-08,5c8e6e02-c46a-4cb4-bb24-86b35a5c526c,S2A_MSIL1C_20181108T153611_N0207_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
3,2018-11-03,c3fb69a6-b19f-4b23-b7b3-543df97a55b8,S2B_MSIL1C_20181103T153619_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
4,2016-05-19,427b931b-a493-4320-9812-28c504d07921,S2A_MSIL1C_20160519T153122_N0202_R025_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
5,2016-05-22,005b7eb7-e993-44ae-8a44-2fd0721ea2da,S2A_MSIL1C_20160522T153622_N0202_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
6,2018-10-29,c4ac45cb-4572-403e-a0a2-77ee8f8470a8,S2A_MSIL1C_20181029T153621_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
7,2016-06-08,3f12d561-9363-4b8b-821b-0359641410fe,S2A_MSIL1C_20160608T153122_N0202_R025_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
8,2018-10-24,a10bd66e-88fe-4a9a-a7be-9f9b72f1df5a,S2B_MSIL1C_20181024T153619_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
9,2016-06-11,ec052ef1-0467-4e67-8786-e9a7095e9a29,S2A_MSIL1C_20160611T153622_N0202_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...


In [0]:
#Find all scenes within 4 days of a fieldwork date
#Make one column for every FW date
FW_df = pd.read_csv('/Volumes/Gibbs_Drive/Sentinel_Test/FW_Dates.csv', parse_dates=[0])

In [0]:
#Define date window
day_window = 4
FW_min = min(FW_df['DATE']).to_pydatetime() - timedelta(days=day_window)
FW_max = max(FW_df['DATE']).to_pydatetime() + timedelta(days=day_window)


def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
  
csv_data = list()

for fw_date in FW_df['DATE']:
  S2_Near = nearest(list(S2_df['DATE']), fw_date)
  S2_Num = (S2_Near - fw_date).days
  S2_ID = S2_df[S2_df['DATE'] == S2_Near].iloc[0,1]
  S2_Title = S2_df[S2_df['DATE'] == S2_Near].iloc[0,2]
  S2_Url = S2_df[S2_df['DATE'] == S2_Near].iloc[0,3]
  csv_data.append((fw_date, S2_Near, S2_Num, S2_ID, S2_Title, S2_Url))

Nearest_DF = pd.DataFrame(csv_data)
Nearest_DF.columns = ['FW', 'S2_DATE', 'S2_NUM', 'S2_ID', 'S2_TITLE', 'S2_QL_URL']
Nearest_DF
#Record of the closes S2 scenes to each FW date

,FW,S2_DATE,S2_NUM,S2_ID,S2_TITLE,S2_QL_URL
0,2018-07-30,2018-07-26,-4,8f5a4b3b-1853-4921-9bac-a79318aea15b,S2B_MSIL1C_20180726T153619_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
1,2018-08-17,2018-08-20,3,dd45f683-e9e3-4689-90f9-813f1b610c11,S2A_MSIL1C_20180820T153621_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
2,2018-08-24,2018-08-20,-4,dd45f683-e9e3-4689-90f9-813f1b610c11,S2A_MSIL1C_20180820T153621_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
3,2018-09-21,2018-09-24,3,1acdec9c-753a-4b1c-b86b-c12da4539d87,S2B_MSIL1C_20180924T153609_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
4,2018-10-05,2018-10-09,4,3c1c8bef-4820-4b86-bdfd-a380578511b9,S2A_MSIL1C_20181009T153621_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
5,2018-10-26,2018-10-29,3,c4ac45cb-4572-403e-a0a2-77ee8f8470a8,S2A_MSIL1C_20181029T153621_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
6,2018-11-02,2018-10-29,-4,c4ac45cb-4572-403e-a0a2-77ee8f8470a8,S2A_MSIL1C_20181029T153621_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
7,2018-11-09,2018-10-29,-11,c4ac45cb-4572-403e-a0a2-77ee8f8470a8,S2A_MSIL1C_20181029T153621_N0206_R068_T18NUJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...


In [0]:
path = '/Volumes/Gibbs_Drive/S2_Quicklooks/'
os.chdir(path)

names = S2_df['NAME']

for i, url in enumerate(S2_df['QL_URL']):
  file_name = path + names[i] + '.jpg'
  print(url)
  print(file_name)
  r = requests.get(url, auth=('hamishgibbs', 'Hamish123!'), stream=True)
  if r.status_code == 200:
    with open(file_name, 'wb') as f:
      for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)#print('here2')

https://scihub.copernicus.eu/dhus/odata/v1/Products('671de52d-c919-46dd-87cd-22dbb2ed0c9d')/Products('Quicklook')/$value
/Volumes/Gibbs_Drive/S2_Quicklooks/S2A_MSIL1C_20160502T153622_N0201_R068_T18NUJ_20160502T154137.jpg
https://scihub.copernicus.eu/dhus/odata/v1/Products('bf7b165f-e3fc-4fbe-bbc4-64b63ca66d92')/Products('Quicklook')/$value
/Volumes/Gibbs_Drive/S2_Quicklooks/S2A_MSIL1C_20160509T152642_N0202_R025_T18NUJ_20160509T152952.jpg
https://scihub.copernicus.eu/dhus/odata/v1/Products('5c8e6e02-c46a-4cb4-bb24-86b35a5c526c')/Products('Quicklook')/$value
/Volumes/Gibbs_Drive/S2_Quicklooks/S2A_MSIL1C_20181108T153611_N0207_R068_T18NUJ_20181108T191115.jpg
https://scihub.copernicus.eu/dhus/odata/v1/Products('c3fb69a6-b19f-4b23-b7b3-543df97a55b8')/Products('Quicklook')/$value
/Volumes/Gibbs_Drive/S2_Quicklooks/S2B_MSIL1C_20181103T153619_N0206_R068_T18NUJ_20181103T190020.jpg
https://scihub.copernicus.eu/dhus/odata/v1/Products('427b931b-a493-4320-9812-28c504d07921')/Products('Quicklook')/$v

In [0]:
#could try third round
confirmed_scenes = [
    'S2A_MSIL1C_20180810T153621_N0206_R068_T18NUJ_20180810T210403',
    'S2A_MSIL1C_20180820T153621_N0206_R068_T18NUJ_20180820T210738',
    'S2A_MSIL1C_20180929T153621_N0206_R068_T18NUJ_20180929T190131']

cloud_scenes = [
    'S2A_MSIL1C_20180731T153621_N0206_R068_T18NUJ_20180731T202224', 
    'S2A_MSIL1C_20180919T153611_N0206_R068_T18NUJ_20180919T202812', 
    'S2A_MSIL1C_20181108T153611_N0207_R068_T18NUJ_20181108T191115', 
    'S2B_MSIL1C_20180815T153619_N0206_R068_T18NUJ_20180815T203026', 
    'S2B_MSIL1C_20180825T153609_N0206_R068_T18NUJ_20180825T205020', 
    'S2B_MSIL1C_20181004T153619_N0206_R068_T18NUJ_20181006T104529', 
    'S2B_MSIL1C_20181024T153619_N0206_R068_T18NUJ_20181024T203259', 
    'S2B_MSIL1C_20181103T153619_N0206_R068_T18NUJ_20181103T190020']
cloud_scenes2 = 

In [0]:
FW_Match = [None] * len(cloud_scenes)
FW_Match


[None, None, None, None, None, None, None, None]

In [0]:
#Get fieldword dates from Nearest_df remove these and make a new FW_df, repeat
#for scene in possible_scenes:
for i, scene in enumerate(confirmed_scenes):
  try:
    FW_Match[i] = Nearest_DF[Nearest_DF['S2_TITLE'] == scene].iloc[0,0]
  except:
    print('No Data ' + str(i))

#Make a new S2_df with all tested scenes removed
#The fieldwork date that has a possible scene has to be removed and the discarded scenes have to be removed
print(len(S2_df))
for scene in Nearest_DF['S2_TITLE']:
    S2_df = S2_df.drop(S2_df[S2_df['NAME'] == scene].index, axis = 0)
print(len(S2_df))
    
#Make a new FW_DF with these dates removed
print(len(FW_df))
for date in FW_Match:
  FW_df = FW_df.drop(FW_df[FW_df['DATE'] == date].index, axis = 0)

print(len(FW_df))
FW_df

No Data 1
37
27
10
8


,DATE
0,2018-07-30
2,2018-08-17
3,2018-08-24
4,2018-09-21
6,2018-10-05
7,2018-10-26
8,2018-11-02
9,2018-11-09


In [0]:
#Field work date for new S2 scene
Nearest_DF[Nearest_DF['S2_TITLE'] == 'S2A_MSIL1C_20180810T153621_N0206_R068_T18NUJ_20180810T210403'].iloc[0,0]

#Now, find corresponding S1 Scene

Timestamp('2018-08-10 00:00:00')

In [0]:
scenes = ['S2A_MSIL1C_20160502T153622_N0201_R068_T18NUJ_20160502T154137',
          'S2A_MSIL1C_20160522T153622_N0202_R068_T18NUJ_20160522T153624',
          'S2A_MSIL1C_20180919T153611_N0206_R068_T18NUJ_20180919T202812',
          'S2A_MSIL1C_20181108T153611_N0207_R068_T18NUJ_20181108T191115',
          'S2B_MSIL1C_20180825T153609_N0206_R068_T18NUJ_20180825T205020',
          'S2B_MSIL1C_20180904T153609_N0206_R068_T18NUJ_20180904T205707',
          'S2B_MSIL1C_20180924T153609_N0206_R068_T18NUJ_20180924T210234',
          'S2B_MSIL1C_20180924T153609_N0206_R068_T18NUJ_20180924T215952',
          'S2B_MSIL1C_20181004T153619_N0206_R068_T18NUJ_20181006T104529',
          'S2B_MSIL1C_20181014T153619_N0206_R068_T18NUJ_20181014T202801',
          'S2B_MSIL1C_20181024T153619_N0206_R068_T18NUJ_20181024T203259',
          'S2B_MSIL1C_20181103T153619_N0206_R068_T18NUJ_20181103T190020']

dl_ids = [None] * len(scenes)

for i, scene in enumerate(scenes):
  dl_ids[i] = S2_df[S2_df['NAME'] == scene].iloc[0,1]

dl_ids
  


['671de52d-c919-46dd-87cd-22dbb2ed0c9d',
 '005b7eb7-e993-44ae-8a44-2fd0721ea2da',
 '393a2a78-9803-422d-9bbf-a3785e6ac69d',
 '5c8e6e02-c46a-4cb4-bb24-86b35a5c526c',
 '6c40ccdd-b97a-4151-b652-1051b0e827ce',
 'a315f647-be81-40cb-8b91-02d338be88cf',
 '1acdec9c-753a-4b1c-b86b-c12da4539d87',
 '014bb8b2-b493-40d6-b211-acc61e894fb2',
 '04eda480-c03f-4af6-8d26-67135af7de7e',
 '93b36f7a-a715-495b-98a0-c2f426ffd50e',
 'a10bd66e-88fe-4a9a-a7be-9f9b72f1df5a',
 'c3fb69a6-b19f-4b23-b7b3-543df97a55b8']

In [0]:
durl = 'https://scihub.copernicus.eu/apihub/odata/v1/Products('
id = "'4eac2f2c-faed-4557-8883-849224120760'" + ')' + '/$value'
#id =  first_scene + ')' + '/$value'

dwurl = durl + id
dwurl


"https://scihub.copernicus.eu/apihub/odata/v1/Products('4eac2f2c-faed-4557-8883-849224120760')/$value"

In [0]:
scene = requests.get(dwurl, auth=('hamishgibbs', 'Hamish123!'))

FileNotFoundError: ignored

In [0]:
open('4eac2f2c-faed-4557-8883-849224120760.zip', 'w').write(scene.content)

TypeError: ignored

In [0]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return local_filename